# Assigment 4

In [1]:
import math
import numpy as np
import scipy.sparse as sp
import igl
import meshplot as mp

from math import sqrt

In [2]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

In [3]:
def find_root(c0,n):
    M = np.zeros((n,n),dtype=complex)
    for i in range(1,n):
        M[i,i-1] = 1.0
    M[0,n-1] = c0
    w, v = np.linalg.eig(M)
    return w[0]

In [4]:
def align_field(V, F, TT, soft_id, soft_value, llambda):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
    
    sqrtl = sqrt(llambda)
    
    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    
    # Rhs of the system
    b = np.zeros(index + soft_id.shape[0], dtype=np.complex128)
    
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        
        data.append(sqrtl); ii.append(index); jj.append(f)
        b[index] = c * sqrtl
        
        index += 1
    
    assert(b.shape[0] == index)
    
    
    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    u = sp.linalg.spsolve(A.T.conjugate() @ A, A.T.conjugate() @ b)
    
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R


In [5]:
def plot_mesh_field(V, F, R, constrain_faces, n):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    
    # Convert R to an angle
    angles = np.arctan2(np.sum(R*T2,axis=1),np.sum(R*T1,axis=1))
    R = R / np.linalg.norm(R, axis=1)[:, None]

    
    for i in range(n):
        R2 = np.cos(angles+i*2*math.pi/n)[:,None] * T1 + np.sin(angles+i*2*math.pi/n)[:,None] * T2
        p.add_lines(B, B + R2 * avg)
    
    return p

In [6]:
R = align_field(v, f, tt, cf, c, 1e6)
plot_mesh_field(v, f, R, cf, 1)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Part 1: Tangent vector fields for scalar field design

In [7]:
def align_field_hard_constraint(V, F, TT, soft_id, soft_value, llambda, n):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])
    assert(n>0)
    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
       
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            ef = np.power(ef,n)
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            eg = np.power(eg,n)
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1

    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).tocsr()
    Q = A.conj().T @ A

    # now we partition the face indices bewteen constraints and actual optimizable index values. 
    all_ids = np.arange(F.shape[0])
    free_ids = np.setdiff1d(all_ids, soft_id) 
    constrained_ids = soft_id 

    # constraint vector u_c
    u_c = np.zeros(len(soft_id), dtype=complex)

    
    for i, f in enumerate(soft_id):
        v = soft_value[i]
        c = np.dot(v, T1[f]) + 1j * np.dot(v, T2[f])
        u_c[i]= c ** n

    #obtain the Qff from the Q matrix : extract diagonal of free ff id's
    Q_ff= Q[free_ids,:][:, free_ids]
    Q_fc= Q[free_ids,:][:, constrained_ids]
    rhs= -Q_fc @ u_c
    #solve the system!
    u_f = sp.linalg.spsolve(Q_ff, rhs)

    # u contains both uf that we just found and the constraints
    u = np.zeros(F.shape[0], dtype=complex)
    u[free_ids] = u_f
    u[constrained_ids] = u_c

    R = np.zeros_like(T1)
    
    for j in range(u.shape[0]):
        u[j] = find_root(u[j],n)
    
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]
    R=R

    return R

RQ = align_field_hard_constraint(v, f, tt, cf, c, 1e6, 1)
print('Vector field with hard constraints:')
plot_mesh_field(v, f, RQ, cf, 1)



Vector field with hard constraints:


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Part 2: Reconstructing a scalar field from a vector field
Find a scalar function S(x) defined over the surface whose gradient fits a given
vector field as closely as possible.

In [8]:
def plot_mesh_field_with_scalar(v, f, Gs_mat, cf, n, s):
  
    col = -s[f].mean(axis=1).flatten()

    B = igl.barycenter(v,f)


    avg = igl.avg_edge_length(v,f) / 2

    R_normalized = Gs_mat / np.linalg.norm( Gs_mat, axis=1)[:, None]
    arrows = B + R_normalized * avg

    p = mp.plot(v, f, c=col, shading={"wireframe": False})

    # Plot vector field
    p.add_lines(B, arrows, shading={"line_color": "black"})

    return p




In [9]:

v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
## scalar field: 
G = igl.grad(v, f)
A = 0.5 * igl.doublearea(v, f)
A_diag = sp.diags(np.repeat(A, 3))
u_flat = R.T.reshape(R.size, 1)

#  solve (G^T A G) s = -2 G^T A u
lhs = G.T @ A_diag @ G
rhs = -2 * G.T @ A_diag @ u_flat

lhs = lhs.tolil()
#  modify s[fixed_v] = 0 
fixed_v = 0
lhs[fixed_v, :] = 0  # zero entire row
lhs[fixed_v, fixed_v] = 1  #set diagonal to 1
rhs[fixed_v] = 0  # update rhs entry here
#back to CSC 
lhs = lhs.tocsc()

s = sp.linalg.spsolve(lhs, rhs)


#NOW PLOT GRADIENT FIELD with scalar field :
# reconsruct gradient field
gs = G @ s
Gs_mat = gs.reshape(3, f.shape[0]).T
print('Vector field from calculated scalar field:')
plot_mesh_field_with_scalar(v, f, Gs_mat, cf, 1, s)

#make scalar field from recnstructed vector field


Vector field from calculated scalar field:


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [10]:
#RECONSTRUCTION ERROR
import matplotlib.pyplot as plt

error_vectors = Gs_mat - R
error = np.linalg.norm(error_vectors, axis=1)**2
print('Map of Poisson parametrization error: error between the gradient of my computed scalar field and the target vector field')
mp.plot(v, f, c=error, shading={"wireframe": False})


print('Dump of S: ' ,s[:50])


Map of Poisson parametrization error: error between the gradient of my computed scalar field and the target vector field


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Dump of S:  [-0.          1.67583695 -1.76469993 -1.85885801  0.02047839  0.20236464
 -0.17633152  0.02820092 -2.08938657 -0.15842508 -0.61604168 -1.93225376
 -1.1274678  -0.67465842 -0.43523509  0.88456311  0.7321628  -0.09492589
 -0.2915436   0.11396174 -0.8993885  -0.57888821 -1.10747115 -0.27514291
 -1.97328822 -1.87256568 -2.05761139 -0.94078959 -1.12221817 -0.98741711
 -0.54536379 -1.42264934 -1.10893013 -0.30430452 -0.25078107  0.09661135
 -0.00439762  0.36128232 -0.18012787 -0.34064863 -0.07270024  0.20334759
  0.38913794  1.30633938 -0.18131551 -0.15205893 -0.21394398 -0.33669563
 -0.13994289  0.41973966]


## Part 3: 

Expermenting with harmonic and LSCM parametrization:

In [11]:
v, f  = igl.read_triangle_mesh("data/camel_head.off")
bound = igl.boundary_loop(f)
bound_uv = igl.map_vertices_to_circle(v, bound)

# harmonic parameterization 
uv = igl.harmonic(v, f, bnd, bound_uv, 1)
uv = np.array(uv)

p1 = mp.subplot(v, f, uv=uv, s=[1, 2, 0])
mp.subplot(uv, f, shading={"wireframe": True}, data=p1, s=[1, 2, 1])

#gradient of scalar field 
v_vals = uv[:, 1]
G = igl.grad(v, f)                
grad_v = (G @ v_vals).reshape(3,f.shape[0]).T 

face_centers = np.mean(v[f], axis=1)
grad_mag = np.linalg.norm(grad_v, axis=1)

#gradient magnitude as face color
plot = mp.plot(v, f, c=grad_mag, return_plot=True, shading={"wireframe": False})

scale = 0.01
arrow_ends = face_centers + scale * grad_v
plot.add_lines(face_centers, arrow_ends, shading={"line_color": "red"})


NameError: name 'bnd' is not defined

In [ ]:
v, f = igl.read_triangle_mesh( "data/camel_head.off")
#boundary, fixed points
b= np.array([1,10])
bound= igl.boundary_loop(f)
b[0] = bound[0]
b[1] = bound[int(bnd.size / 2)]
bc = np.array([[0.0, 0.0], [1.0, 0.0]])

#computing lscm parametrizaton 
_, uv_lscm = igl.lscm(v, f, b, bc)

p = mp.subplot(v, f, uv=uv_lscm, shading={"wireframe": False, "flat": False}, s=[1, 2, 0])
mp.subplot(uv_lscm, f,  shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p)


In [ ]:

def scalar_field(v, f):
    G = igl.grad(v, f)
    A = 0.5 * igl.doublearea(v, f)
    A_diag = sp.diags(np.repeat(A, 3))
    u_flat = R.T.reshape(R.size, 1)
    
    #  solve (G^T A G) s = -2 G^T A u
    lhs = G.T @ A_diag @ G
    rhs = -2 * G.T @ A_diag @ u_flat
    
    lhs = lhs.tolil()
    #  modify s[fixed_v] = 0 
    fixed_v = 0
    lhs[fixed_v, :] = 0  # zero entire row
    lhs[fixed_v, fixed_v] = 1  #set diagonal to 1
    rhs[fixed_v] = 0  # update rhs entry here
    #back to CSC 
    lhs = lhs.tocsc()

    s = sp.linalg.spsolve(lhs, rhs)
    
    gs= G @ s
    Gs_mat = gs.reshape(3, f.shape[0]).T
    

    
    return s, gs





## PART 4: Editing parametrization with vector fields

Starting with a harmonic/LSCM parameterization, use the results of the previous steps to replace one of the U , V functions with a function obtained from a smooth user-guided vector field. Visualize the resulting U or V replacement function and its gradient atop the mesh, and texture the mesh with the new parameterization.



* “To edit the parameterization, I define a scalar function by reconstructing it from a smooth, user-guided vector field.
* This function is then used to replace one coordinate (typically V) of the original UV parameterization.”

In [12]:

def edit_param(v, f, tt, cf, c):
    # smooth user-guided vector field
    R = align_field_hard_constraint(v, f, tt, cf, c, 1e6, 1) 
    #reconstruct scalar function
    s, Gs = scalar_field(v, f)
    G = igl.grad(v, f)
    
    # subsitute V by the gradient of the user-guided vector field
    bnd = igl.boundary_loop(f)
    bound_uv = igl.map_vertices_to_circle(v, bnd)

    # harmonic parameterization 
    uv = igl.harmonic(v, f, bnd, bound_uv, 1)
    
    #REPLACE V WITH RECONSTRUCTED SCALAR
    uv = np.vstack((uv[:,0], s)).T
    return uv


In [123]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)
c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]


uv= edit_param(v, f, tt, cf, c)
# replace one of the U, V functions with a function obtained from a smooth user-guided vector field
# plot mesh with uv
avg = igl.avg_edge_length(v, f)/2
B = igl.barycenter(v, f)
G = igl.grad(v, f)
gs = G @ s
Gs_mat = gs.reshape(3, f.shape[0]).T

p = mp.plot(v, f, uv=uv, shading={"wireframe": True})
p.add_lines(B, B + Gs_mat*avg, shading={"line_color": "cyan"})

#here we can see the distoriotion!
p_distortion= mp.plot(uv, f, uv=uv, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, -0.2…

- To detect flipped faces, we compare the orientation of each triangle in the original 3D mesh and its corresponding representation in UV space. 
- Specifically, we compute the normal vector of each triangle using the cross product of its edge vectors.
- In the UV plane, if the signed area, z in cross product, becomes negative it means the triangle's orientation is reversed indicating the flipped face.
- Flipped triangle has had a rotation greater than 180 degrees, which breaks the correct surface orientation in parameterization


In [124]:
def detect_flipped_faces(v, f, uv):
    flipped_faces = []
    for i, face in enumerate(f):
        #vertices and edges per face
        x1, x2, x3 = v[face]
        e1= x2- x1
        e2 = x3- x1
        normal = np.cross(e1, e2) #ORIGINAL NORMAL

        # UV coordinates 
        uv1, uv2, uv3 = uv[face]
        e1_uv= np.append(uv2 - uv1, 0.0)
        e2_uv= np.append(uv3 - uv1, 0.0)
        normal_uv = np.cross(e1_uv, e2_uv)

        # IMPORTANT: COMPUTE ANGLE BETWEEN NORMALS
        dot = np.dot(normal, normal_uv)
        norm_product = np.linalg.norm(normal) * np.linalg.norm(normal_uv)
        angle = np.arccos(np.clip(dot / norm_product, -1.0, 1.0))
        
        if angle > np.pi / 2: #180 degrees
            flipped_faces.append(i)

    return flipped_faces


flipped = detect_flipped_faces(v, f, uv)
col = np.zeros_like(f)
col[flipped] =1
p = mp.plot(v, f, c=col, shading={"wireframe": True})
print('ascii dump of the flipped indices!', flipped)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

ascii dump of the flipped indices! [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 90, 91, 95, 96, 97, 98, 99, 100, 101, 102, 103, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223,